In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:09<01:24,  2.15it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:42<00:34,  2.91it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 64%|██████▎   | 127/200 [00:52<00:26,  2.74it/s]

In [ ]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [24]:
model = torch_classes.GRUNetV2(12,64,num_layers=2).to(device='cuda:0')

In [25]:
model

GRUNetV2(
  (gru): GRU(12, 64, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): LayerNorm((12800,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=12800, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)

In [ ]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNetV2(12,config['hidden_size'],num_layers=config['num_layers']).to('cuda:0')
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        print(model)
        trading_df.reset_hidden(config['hidden_size'],num_layers=config['num_layers'])
        criterion = nn.MSELoss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [11]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [12]:
config_static = {'learning_rate':0.0002, 'hidden_size':128, 'num_layers':2}

In [13]:
trading_data.reset_hidden(128,2)
trading_data.detach_hidden()

In [14]:
model

GRUNetV2(
  (gru): GRU(12, 64, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): LayerNorm((12800,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=12800, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=640, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)

In [15]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]]
X = trading_data.packed_val_x[0] 
Y = trading_data.packed_val_y[0].data

In [16]:
# hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

In [17]:
# hidden_in 

In [18]:
# output,hidden = model(X,hidden_in,p1=True)

In [19]:
# model = torch_classes.GRUNetV2(12,64)
# model_loc = f"C:/Users/Nick/Documents/GitHub/OptiverKaggle/models/tough-totem-105/tough-totem-105_160.pt"
# model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
# print(model_data['model_state_dict'].keys())

# print(model_data['model_state_dict'].keys())
# del_keys = ['fc0.weight', 'fc0.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias']
# [model_data['model_state_dict'].pop(k) for k in del_keys]
# model.load_state_dict(model_data['model_state_dict'], strict=False)
# model = model.to(device='cuda:0')

In [ ]:
model = model_pipeline(trading_data, config_static)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


GRUNetV2(
  (gru): GRU(12, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): LayerNorm((25600,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=25600, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=640, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)


  0%|          | 0/10000 [00:00<?, ?it/s]

{'epoch_loss': tensor(1.2448, device='cuda:0'), 'epoch_l1_loss': tensor(1.3582, device='cuda:0'), 'epoch': 0}
   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.055811
1      1    0     0 -5.519986  0.027119
2      2    0     0 -8.389950 -0.087402
3      3    0     0 -4.010201 -0.104228
4      4    0     0 -7.349849 -0.115217
5      5    0     0  6.779432 -0.042466
6      6    0     0 -2.499819  0.022411
7      7    0     0 -1.959801  0.018460
8      8    0     0 -5.970001 -0.114398
9      9    0     0  7.970333 -0.086361
created path
{'epoch_loss': tensor(1.2445, device='cuda:0'), 'epoch_l1_loss': tensor(1.3578, device='cuda:0'), 'epoch': 1}
{'epoch_loss': tensor(1.2445, device='cuda:0'), 'epoch_l1_loss': tensor(1.3579, device='cuda:0'), 'epoch': 2}
{'epoch_loss': tensor(1.2446, device='cuda:0'), 'epoch_l1_loss': tensor(1.3580, device='cuda:0'), 'epoch': 3}
{'epoch_loss': tensor(1.2445, device='cuda:0'), 'epoch_l1_loss': tensor(1.3580, device='cuda:0'), 'epoch': 

Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_18348\3059119270.py", line 12, in model_pipeline
    training_testing.train_model(trading_df,model,config,optimizer,criterion)
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\training_testing.py", line 59, in train_model
    loss.backward()
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\autograd\__init__.py", line 200, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch_l1_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂█▁▁▅▄▁▁▁▁▁▁
epoch_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂█▁▁▅▄▁▁▁▁▁▁
loss_1,█▃▁▁█▃▃▁▁█▃▃▁▁█▃▃▁██▃▃▁██▃▃▁██▃▁▁██▃▁▁██
relu,▁▄▅▄▄▆▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
val_epoch_loss,█▄▂▃▃▂▄▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▁▂▂▂▂▂▂▁▂▁▂
val_epoch_loss_l1,█▆▁▄▅▃▇▃▄▂▅▄▂▅▃▄▄▆▂▅▃▃▅▅▅▂▆▂▅▃▄▃▄▄▅▄▃▅▄▄
val_relu,▁▄▅▅▅▆▅▅▅▆▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇████████████
epoch,124
epoch_loss,5.93973
loss_1,6.36165


KeyboardInterrupt: 